# Split Apply Combine

Often times, we will want to split a dataset based on a key value in a column, and do something within this group, and return this information for each group.  We will often see this referred to as the **split-apply-combine** paradigm.  With Pandas, we will often use the `groupby` method to carry out the splitting piece of this. 

**OBJECTIVES**:

- Use `groupby` to split dataset into groups, and groups within groups
- Apply different methods to `groupby` objects
- Use aggregation methods on `groupby` objects
- Write and use functions of our own on `groupby` objects with `apply`

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
df = pd.DataFrame({'street': ['a', 'a', 'b', 'b', 'a'],
                  'agent': ['one', 'one', 'two', 'one', 'two'],
                  'sq_ft': np.random.randint(1000, 4500, 5),
                  'price': np.random.randint(100000, 600000, 5)})

In [3]:
df.head()

,street,agent,sq_ft,price
0,a,one,3647,144087
1,a,one,2940,159309
2,b,two,2859,253947
3,b,one,3359,594042
4,a,two,4220,458649


### `groupby` Basics

To begin, it is important to recognize that the result of using the `groupby` method is a `groupby` object.  This is an object that has simply grouped the data according to our input.  From here, we can select columns like usual and apply basic quantitative methods.  If we wanted to know the average square footage by agent in our mini-data above, we select the column we are concerned with (`df['sq_ft']`), and group this by agent(`.groupby(df['agent'])`).  We are returned an object for which we can then apply the mean to.  Below, we save our grouped data as `grouped`, and can subsequently perform a variety of methods.

In [5]:
df['sq_ft'].groupby(df['agent'])

In [6]:
grouped = df['sq_ft'].groupby(df['agent'])

In [7]:
grouped.mean()

agent
one    3315.333333
two    3539.500000
Name: sq_ft, dtype: float64

In [8]:
grouped.count()

agent
one    3
two    2
Name: sq_ft, dtype: int64

In [9]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
agent,,,,,,,,
one,3.0,3315.333333,355.516994,2940.0,3149.50,3359.0,3503.00,3647.0
two,2.0,3539.500000,962.372329,2859.0,3199.25,3539.5,3879.75,4220.0


We can pass more than one value to group by, and will be returned an object that has two levels of indices.  For example, if we wanted to know:

$$
\textit{What is the average price for each agent by street?}
$$

We will select the price column, group the data by agent and street, and apply the mean method to this.  Notice that there are two levels of indices, *agent* and *street*.  We can return a dataframe that unstacks the levels with the `unstack` method.  

In [10]:
means = df['price'].groupby([df['agent'], df['street']]).mean()

In [11]:
means

agent  street
one    a         151698
       b         594042
two    a         458649
       b         253947
Name: price, dtype: int64

In [12]:
means.keys()

MultiIndex(levels=[['one', 'two'], ['a', 'b']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['agent', 'street'])

In [13]:
means.unstack()

street,a,b
agent,,
one,151698,594042
two,458649,253947


In [14]:
means[0]

151698

In [15]:
means[3]

253947

In [17]:
means.shape

(4,)

### Example I 

Using our tips dataset example, we can explore a few straightforward questions using `groupby()`.  

1. What is the average tip by gender?
2. What is the max tip on each day?
3. What is the min tip on each day by time?
4. What is the average bill by party size?

In [18]:
tips = sns.load_dataset('tips')

In [21]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [23]:
tips.groupby('sex')['tip'].mean()

sex
Male      3.089618
Female    2.833448
Name: tip, dtype: float64

In [24]:
gender_groups = tips.groupby('sex')
print(gender_groups)

In [27]:
print(gender_groups)

In [38]:
tips.groupby('day').max()

,total_bill,tip,size
day,,,
Thur,43.11,6.70,6
Fri,40.17,4.73,4
Sat,50.81,10.00,5
Sun,48.17,6.50,6


In [36]:
day_group = tips.groupby('day')
time_group = tips.groupby('time')

In [40]:
tips.groupby(['day','time'])(['tip']).min()

TypeError: 'DataFrameGroupBy' object is not callable

In [35]:
day_time_group = tips.groupby(df[day_group],df[time_group]).min()

KeyError: <pandas.core.groupby.groupby.DataFrameGroupBy object at 0x1a179369b0>

In [ ]:
tips.groupby('sex')[['tip']].mean()

In [ ]:
tips.groupby('day')[['tip']].max()

In [ ]:
tips.groupby(['day', 'time'])[['tip']].min()

In [ ]:
tips.groupby(['size', 'day'])[[ 'tip']].mean().unstack()

Average bill and tip by smoker and time.

In [ ]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean()

In [ ]:
tips.groupby(['smoker', 'time'])[['total_bill', 'tip']].mean().unstack()

The results of the groupby object are iterable.  For example, if we group the tips data by smoker, we get back the group and the elements of this group.  For multiple indicies of groups, we pass these as tuples.

In [ ]:
for name, group in tips.groupby(['smoker']):
    print(name)
    print(group)

In [ ]:
for (n1, n2), group in tips.groupby(['sex', 'smoker']):
    print(n1, n2)
    print(group)

In [ ]:
#one-liner to create dictionary of groups
pieces = dict(list(tips.groupby('sex')))

In [ ]:
pieces['Male'].head()

In [ ]:
pieces['Female'].head()

### Data Aggregation

Here, we are interested in using the aggreate function to apply functions that we have developed which return scalar values.  For example, we can write a function that determines the range of tips on a given day.  We are taking all of the values for tips by group, and finding a single value for this.  Similarly, we could do something like investigate the quantiles of each group.

In [41]:
grouped = tips.groupby('day')

In [42]:
def ranger(df):
    return df.max() - df.min()

In [43]:
grouped.agg(ranger)

,total_bill,tip,size
day,,,
Thur,35.60,5.45,5
Fri,34.42,3.73,3
Sat,47.74,9.00,4
Sun,40.92,5.49,4


In [44]:
grouped.quantile(0.7)

0.7,size,tip,total_bill
day,,,
Thur,2.0,3.000,18.990
Fri,2.0,3.150,19.134
Sat,3.0,3.164,22.470
Sun,3.0,3.695,24.570


In [45]:
grouped.quantile(0.9)['tip']

day
Thur    4.920
Fri     4.060
Sat     4.802
Sun     5.035
Name: tip, dtype: float64

We can add a tip percentage column as follows.

In [46]:
tips['pct_tip'] = tips['tip']/tips['total_bill']

In [47]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size,pct_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [48]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg('mean')

sex     smoker
Male    Yes       0.152771
        No        0.160669
Female  Yes       0.182150
        No        0.156921
Name: pct_tip, dtype: float64

In [49]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(['mean', 'std', ranger])

mean       std    ranger
sex    smoker                              
Male   Yes     0.152771  0.090588  0.674707
       No      0.160669  0.041849  0.220186
Female Yes     0.182150  0.071595  0.360233
       No      0.156921  0.036421  0.195876

In [50]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg([('Average Tipping Percentage', 'mean'), ('Standard Deviation', 'std'), ('Range', ranger)])

Average Tipping Percentage  Standard Deviation     Range
sex    smoker                                                          
Male   Yes                       0.152771            0.090588  0.674707
       No                        0.160669            0.041849  0.220186
Female Yes                       0.182150            0.071595  0.360233
       No                        0.156921            0.036421  0.195876

In [52]:
funcs = ['mean', 'max', 'min', 'std']

In [53]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(funcs)

mean       max       min       std
sex    smoker                                        
Male   Yes     0.152771  0.710345  0.035638  0.090588
       No      0.160669  0.291990  0.071804  0.041849
Female Yes     0.182150  0.416667  0.056433  0.071595
       No      0.156921  0.252672  0.056797  0.036421

In [54]:
#overwrite results with new
#column names
func_named = [('Average', 'mean'), ('Maximum', 'max')]

In [55]:
tips.groupby(['sex', 'smoker'])['pct_tip'].agg(func_named)

Average   Maximum
sex    smoker                    
Male   Yes     0.152771  0.710345
       No      0.160669  0.291990
Female Yes     0.182150  0.416667
       No      0.156921  0.252672

In [56]:
#pass different aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': np.min, 'pct_tip': np.max})

tip   pct_tip
sex    smoker                
Male   Yes     1.00  0.710345
       No      1.25  0.291990
Female Yes     1.00  0.416667
       No      1.00  0.252672

In [57]:
#pass multiple aggregation functions
tips.groupby(['sex', 'smoker']).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

tip             pct_tip
               amin      mean      amax
sex    smoker                          
Male   Yes     1.00  3.051167  0.710345
       No      1.25  3.113402  0.291990
Female Yes     1.00  2.931515  0.416667
       No      1.00  2.773519  0.252672

In [58]:
#ignore index labels
tips.groupby(['sex', 'smoker'], as_index=False).agg({'tip': [np.min, np.mean], 'pct_tip': np.max})

sex smoker   tip             pct_tip
                  amin      mean      amax
0    Male    Yes  1.00  3.051167  0.710345
1    Male     No  1.25  3.113402  0.291990
2  Female    Yes  1.00  2.931515  0.416667
3  Female     No  1.00  2.773519  0.252672

### Exercise

Using the titanic data, let's use the `groupby` function to provide aggregate information about groups.

1. How many male and female passengers survived and died?
2. How many passengers from each class lived and died?
3. Create a new column that uses the `age` column to create the groups:
  - Children
  - Teenagers
  - Young Adults
  - Middle Aged
  - Older
  - Senior
  
You are free to define these as you see fit.  Further, you should use better and easier labels in your code.  Investigate the survival within each of these groups.

In [59]:
titanic = pd.read_csv('data/eda_data/titanic.csv')

In [60]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
survived    891 non-null int64
pclass      891 non-null int64
name        891 non-null object
sex         891 non-null object
age         714 non-null float64
sibsp       891 non-null int64
parch       891 non-null int64
ticket      891 non-null object
fare        891 non-null float64
cabin       204 non-null object
embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 76.6+ KB


In [61]:
titanic.groupby(['sex','survived'])

In [66]:
sex_surv_group = titanic.groupby(['sex','survived'])

In [67]:
sex_surv_group.describe()

age                                                        \
                 count       mean        std   min    25%   50%    75%   max   
sex    survived                                                                
female 0          64.0  25.046875  13.618591  2.00  16.75  24.5  33.25  57.0   
       1         197.0  28.847716  14.175073  0.75  19.00  28.0  38.00  63.0   
male   0         360.0  31.618056  14.056019  1.00  21.75  29.0  39.25  74.0   
       1          93.0  27.276022  16.504803  0.42  18.00  28.0  36.00  80.0   

                  fare            ...  pclass       sibsp                      \
                 count       mean ...     75%  max  count      mean       std   
sex    survived                   ...                                           
female 0          81.0  23.024385 ...     3.0  3.0   81.0  1.209877  1.814635   
       1         233.0  51.938573 ...     3.0  3.0  233.0  0.515021  0.737533   
male   0         468.0  21.960993 ...     3.0  3.0  468.0  0.440171  1.138507   
       1         109.0  40.821484 ...     3.0  3.0  109.0  0.385321  0.636952   

                                          
                 min  25%  50%  75%  max  
sex    survived                           
female 0         0.0  0.0  1.0  1.0  8.0  
       1         0.0  0.0  0.0  1.0  4.0  
male   0         0.0  0.0  0.0  0.0  8.0  
       1         0.0  0.0  0.0  1.0  4.0  

[4 rows x 40 columns]

In [68]:
titanic.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [69]:
sex_surv_group.head()

,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [70]:
sex_surv_group.max()

age      fare                                          name  \
sex    survived                                                                 
female 0         57.0  151.5500                         Zabour, Miss. Thamine   
       1         63.0  512.3292  de Messemaeker, Mrs. Guillaume Joseph (Emma)   
male   0         74.0  263.0000                   van Melkebeke, Mr. Philemon   
       1         80.0  512.3292                       de Mulder, Mr. Theodore   

                 parch  pclass  sibsp      ticket  
sex    survived                                    
female 0             6       3      8  W./C. 6609  
       1             5       3      4   WE/P 5735  
male   0             5       3      8   WE/P 5735  
       1             2       3      4   SW/PP 751

In [ ]:
titanic.groupby(['sex', 'survived'])['survived'].count()

In [ ]:
titanic.groupby(['pclass', 'survived'])['survived'].count().unstack()

In [ ]:
age_group = []
for age in titanic.age:
    if age <= 12:
        age_group.append('child')
    elif age < 20:
        age_group.append('teenager')
    elif age < 50:
        age_group.append('mid-age')
    elif age < 70:
        age_group.append('older')
    else:
        age_group.append('old')

In [ ]:
titanic['age_group'] = age_group

In [ ]:
titanic.head()

In [ ]:
titanic.groupby(['age_group', 'survived'])['survived'].count()

In [ ]:
titanic.groupby(['age_group', 'survived'])['survived'].count().unstack()

### `apply`

Beyond aggregation functions, we can apply a more general call to functions that don't necessarily return a scalar value.  For example, suppose we wanted to pass a function that will take the top 5 tip percentages.  Then, we can apply this to different groupings of the data.  Because our function takes a column argument as well as a top number, we can call these using the `apply` method also.

In [ ]:
def top(df, n=5, column='pct_tip'):
    return df.sort_values(by = column)[-n:]

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column = 'total_bill')

In [ ]:
tips.groupby('smoker', group_keys = False).apply(top)

In [ ]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = [])

In [ ]:
tips.groupby('smoker')['pct_tip'].describe(percentiles = []).unstack()

### Example for filling `na`

Suppose we have a dataset that we want to use group specific values as replacements for missing values.  In the example below, it seems we have some missing bidderrate values.  Perhaps we make the assumption that an acceptable replacement would be the average bidder rating for each auction.  To apply this, we first split the data and pass an anonymous `lambda` function that will be applied to each group.  In this case, we are filling the `na` values with the `mean`.

In [ ]:
auction = pd.read_csv('data/auction/auction.csv')

In [ ]:
auction.info()

In [ ]:
auction.bidderrate.describe()

In [ ]:
auction.groupby('auctionid').mean().head()

In [ ]:
fill_bidrate = lambda g: g.fillna(g.mean())

In [ ]:
auction_filled = pd.merge(auction, auction.groupby('auctionid').apply(fill_bidrate))

In [ ]:
auction_filled.head()

### Further Reading

- [Pandas `groupby` documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html)
- [Datacamp Article](https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby)